In [93]:
# Author:Ning Pei Ding
# Date: 02/10/2020
import pandas as pd
import numpy as np
import scipy.io
import statsmodels.api as sm

In [150]:
mat = scipy.io.loadmat('china.mat')

/Users/ningpeiding/anaconda3/lib/python3.6/site-packages/scipy/io/matlab/mio.py:142: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [148]:
mret = np.array(mat['china_monthly_ri'])
market = np.array(mat['china_monthly_mv'])
rankrt = np.stack([(mret[:,i] - mret[:,i-1])/mret[:,i-1] for i in range(1,mret.shape[1])], axis = -1)
mk = np.stack([market[:,i] for i in range(1,mret.shape[1])], axis = -1)
momentum = np.stack([np.prod(rankrt[:,i-11:i] + 1,axis = 1) -1 for i in range(11,rankrt.shape[1])], axis = -1)
momentum= np.concatenate((np.zeros([momentum.shape[0],11]), momentum), axis=1)
rf = pd.read_csv('F-F_Research_Data_Factors.CSV')
rf = rf[rf['Time'] >= mat['mdates'][0][0]]['RF']

In [156]:
rankrt_none = rankrt
rankrt_none[np.isnan(rankrt_none)] = 0
mk_none = mk
mk_none[np.isnan(mk_none)] = 0
ew = np.array([np.dot(rankrt_none[:,i], mk_none[:,i-1]/np.nansum(mk_none[:,i-1])) for i in range(1,nc)])

/Users/ningpeiding/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [160]:
rankrt[~np.isnan(rankrt)].mean()

0.004920084428640402

In [165]:
%%time
from sklearn.linear_model import LinearRegression
nc = rankrt.shape[1]
nr = rankrt.shape[0]
rf = pd.read_csv('F-F_Research_Data_Factors.CSV')
rf = rf[rf['Time'] >= mat['mdates'][1][0]]['RF']
China_betas = np.zeros((nr,nc))
for i in range(85,nc):
    print(i)
    for j in range(nr):
        if np.isnan(rankrt[j,i-24:i]).sum() == 0:
#             China_betas[j,i] = sm.OLS(rankrt[j,i-24:i] - rf[i-24:i]/100, sm.add_constant(ew[i-24:i]- rf[i-24:i]/100)).fit().params[1]
            res = LinearRegression().fit(np.array((ew[i-24:i]- rf[i-24:i]/100)).reshape(-1, 1), rankrt[j,i-24:i] - rf[i-24:i]/100)
            China_betas[j,i] = res.coef_[0]
        

85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338

In [166]:
np.savetxt("betas1.csv", China_betas, delimiter=",")

In [88]:
China_betas.shape

(3845, 384)

In [90]:
(China_betas< 0).sum()

1905

In [66]:
3845*384 - np.isnan(rankrt).sum()

525233

In [64]:
rankrt.shape[0]

(3845, 384)

In [77]:
np.isnan(rf).sum()

0